# CS295/395: Secure Distributed Computation
## Homework 4

## Definitions

In [1]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import urllib.request
import galois
GF = galois.GF(2 ** 13 - 1)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the field size and dictionary to hold received messages."""
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

# Generate Shamir shares for secret v with threshold t and number of shares n
def shamir_share(v, t, n):
    coefficients = GF([GF.Random() for _ in range(t-1)] + [v])
    poly = galois.Poly(coefficients)
    shares = [(GF(x), poly(GF(x))) for x in range(1, n+1)]
    return shares

# Reconstruct the secret from at least t Shamir shares
def reconstruct(shares):
    xs = GF([s[0] for s in shares])
    ys = GF([s[1] for s in shares])
    poly = galois.lagrange_poly(xs, ys)
    #print(poly)
    secret = poly(0)
    
    return secret

## Question 1 (20 points)

Describe a protocol to multiply three input numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive one share of each number. Each party should output *one share of the product*, using a $(t, n)$ Shamir secret sharing scheme (i.e. the threshold for the output should be the same as the threshold for the input).

\begin{equation*}
\textbf{Functionality: Multiply Three Numbers}\\
\fbox{$\mathcal{F}(a, b, c) = a \cdot b \cdot c$}
\end{equation*}

**HINT**: reference the in-class exercise from 9/19/2022.

Do a single multiplecation of shamir shares and then deg reduction

- R1
  - Each party $P_i$ recieves shares $a_i$, $b_i$, $c_i$ as input
  - Let $s_i = a_i * b_i$ - threshold for $s_i$ will be less than or equal to $2t$ where t is the initial threshold
    - $s_i$ is exactly $q(\alpha_i)$
  - $P_i$ computes $h_i^1 ... h_i^n$ = `share`($s_i$, t, n)
  - $P_i$ sends share $h_i^j$ to party $j$
- R2
  - Each party $P_i$ recieves the shares $h_j^i$ (yes the sub and superscripts are flipped)
  - $P_i$ computes $\sum_j (h_j^i * \lambda_j)$. This value is the product of $a$ and $b$ but not yet $c$. So then this product is shamir shared with threshold $t$, and similarly to round 1, shares of $c$ are multiplied in and then degree reduction started.
- R3
  - Each party $P_i$ recieves the shares $h_j^i$ (yes the sub and superscripts are flipped)
  - $P_i$ computes $\sum_j (h_j^i * \lambda_j)$. This is the final product

## Question 2

Implement your protocol from question 1.

In [2]:
class MultThreeParty(Party):
    def round1(self, parties, a_shr, b_shr, c_shr, t):
        self.input = (a_shr, b_shr, c_shr)
        self.c_shr = c_shr # save this one for later
        self.parties = parties
        n = len(parties)
        assert t <= n/2
        
        # - Each party $P_i$ recieves shares $a_i$, $b_i$ as input
        # - Let $s_i = a_i * b_i$ - threshold for $s_i$ will be less than or equal to $2t$ where t is the initial threshold
        #     - $s_i$ is exactly $q(\alpha_i)$

        a_x, a_y = a_shr
        b_x, b_y = b_shr
        c_x, c_y = c_shr

        # they better have the same x coord
        assert a_x == b_x == c_x

        s_i = a_y * b_y #q(x_i) higher degree than we'd like (degree 2t at most)
        self.x_coord = a_x # save this for round 2

        # - $P_i$ computes $h_i^1 ... h_i^n$ = `share`($s_i$, t, n)
        h_i_js = shamir_share(s_i, t, n)
        # - $P_i$ sends share $h_i^j$ to party $j$
        for party, share in zip(self.parties, h_i_js):
            self.send(party, 1, share)

    def round2(self):
        n = len(self.parties)
        
        # - Each party $P_i$ recieves the shares $h_j^i$ (yes the sub and superscripts are flipped)
        h_j_is = self.received[1]
        h_j_is_y = [s[1] for s in h_j_is]

        # $P_i$ computes $\sum_j (h_j^i * \lambda_j)$ and ouptuts this value as its own share of the origional product with threshold $t$
        V_a = GF(np.vander(range(1,n+1), increasing=True))
        V_a_inv = np.linalg.inv(V_a)
        lambda_js = V_a_inv[0]

        prods = [h_j_is_y[i] * lambda_js[i] for i in range(n)]

        self.new_share = self.x_coord, GF(prods).sum()

        n_x, n_y = self.new_share
        c_x, c_y = self.c_shr

        s_i = n_y * c_y

        h_i_js = shamir_share(s_i, t, n)

        for party, share in zip(self.parties, h_i_js):
            self.send(party, 2, share)

    def round3(self):
        n = len(self.parties)

        h_j_is = self.received[2]
        h_j_is_y = [s[1] for s in h_j_is]

        # $P_i$ computes $\sum_j (h_j^i * \lambda_j)$ and ouptuts this value as its own share of the origional product with threshold $t$
        V_a = GF(np.vander(range(1,n+1), increasing=True))
        V_a_inv = np.linalg.inv(V_a)
        lambda_js = V_a_inv[0]

        prods = [h_j_is_y[i] * lambda_js[i] for i in range(n)]

        self.output = self.x_coord, GF(prods).sum()

        return self.output

In [3]:
# TEST CASE for question 2

NUM_PARTIES = 6
# (t, n)-Shamir scheme
n = NUM_PARTIES
t = 3

shares1 = shamir_share(5, t, n)
shares2 = shamir_share(6, t, n)
shares3 = shamir_share(7, t, n)

parties = [MultThreeParty() for _ in range(NUM_PARTIES)]

for p,s1,s2,s3 in zip(parties, shares1, shares2, shares3):
    p.round1(parties, s1, s2, s3, t)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    # print(p.get_view())
    print(p.output)

output_shares = [p.output for p in parties]
print('Reconstruction, with all shares:', reconstruct(output_shares))
print('Reconstruction, with 3 shares:', reconstruct(output_shares[:3]))
print('Reconstruction, with 2 shares:', reconstruct(output_shares[:2]))

assert reconstruct(output_shares) == 210
assert reconstruct(output_shares[:3]) == 210
assert reconstruct(output_shares[:2]) != 210

(GF(1, order=8191), GF(2743, order=8191))
(GF(2, order=8191), GF(223, order=8191))
(GF(3, order=8191), GF(841, order=8191))
(GF(4, order=8191), GF(4597, order=8191))
(GF(5, order=8191), GF(3300, order=8191))
(GF(6, order=8191), GF(5141, order=8191))
Reconstruction, with all shares: 210
Reconstruction, with 3 shares: 210
Reconstruction, with 2 shares: 5263


## Question 3

Describe a protocol to compute the product of a list of $k$ numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive a list containing one share of each number. Each party should output *one share of the product*, using a $(t, n)$ Shamir secret sharing scheme (i.e. the threshold for the output should be the same as the threshold for the input).

\begin{equation*}
\textbf{Functionality: Product of $k$ Numbers}\\
\fbox{$\mathcal{F}(x_1, \dots, x_k) = \prod_{i=1}^k x_i$}
\end{equation*}

**HINT**: This problem is intentionally open-ended. The number of rounds of communication between the parties will depend on $k$.

- R1
  - Each party $P_i$ recieves shares $x_1, x_2, ..., x_k$ as input
  - Let $s_i = x_1 * x_2$ - threshold for $s_i$ will be less than or equal to $2t$ where t is the initial threshold
  - $P_i$ computes $h_i^1 ... h_i^n$ = `share`($s_i$, t, n)
  - $P_i$ sends share $h_i^j$ to party $j$
- RN
  - Each party $P_i$ recieves the shares $h_j^i$ (yes the sub and superscripts are flipped)
  - $P_i$ computes $t_i = \sum_j (h_j^i * \lambda_j)$. This value is the product of $x_{n-1} * x_{n-2} * ... * x_1$ (all the previous $x$'s)
    - If all the numbers have been multiplied in, this is our output and we are done
  - $s_i = t_i * x_n$ - threshold for $s_i$ will be less than or equal to $2t$ where t is the initial threshold
  - $P_i$ computes $h_i^1 ... h_i^n$ = `share`($s_i$, t, n)
  - $P_i$ sends share $h_i^j$ to party $j$

## Question 4

Implement your protocol from question 3.

In [4]:
class MultListParty(Party):
    def round1(self, parties, xs, t):
        self.input = xs
        self.xs = xs
        self.parties = parties
        self.is_done = False
        n = len(parties)
        assert t <= n/2

        a_x, a_y = xs[0]
        b_x, b_y = xs[1]

        # they better have the same x coord
        assert a_x == b_x

        s_i = a_y * b_y #q(x_i) higher degree than we'd like (degree 2t at most)
        self.x_coord = a_x # save this for later
        
        h_i_js = shamir_share(s_i, t, n)
  
        for party, share in zip(self.parties, h_i_js):
            self.send(party, 1, share)


    def roundn(self, round):
        n = len(self.parties)

        # - Each party $P_i$ recieves the shares $h_j^i$ (yes the sub and superscripts are flipped)
        h_j_is = self.received[round - 1]
        h_j_is_y = [s[1] for s in h_j_is]

        # $P_i$ computes $\sum_j (h_j^i * \lambda_j)$ and ouptuts this value as its own share of the origional product with threshold $t$
        V_a = GF(np.vander(range(1,n+1), increasing=True))
        V_a_inv = np.linalg.inv(V_a)
        lambda_js = V_a_inv[0]

        prods = [h_j_is_y[i] * lambda_js[i] for i in range(n)]

        self.cummulative_share = self.x_coord, GF(prods).sum()


        if round == len(self.xs):
            self.is_done = True
            self.output = self.cummulative_share
            return

        # add the next term to the cummulative share

        a_x, a_y = self.cummulative_share
        b_x, b_y = self.xs[round]

        # they better have the same x coord
        assert a_x == b_x

        s_i = a_y * b_y #q(x_i) higher degree than we'd like (degree 2t at most)

        h_i_js = shamir_share(s_i, t, n)

        for party, share in zip(self.parties, h_i_js):
            self.send(party, round, share)

    

In [5]:
# Driver function for question 4
# NOTE: you can modify this function, if it helps for your implementation
# You may also want to uncomment pieces of the function for debugging
def run_list_prod():
    NUM_PARTIES = 6
    # (t, n)-Shamir scheme
    n = NUM_PARTIES
    t = 3
    k = 5

    parties = [MultListParty() for _ in range(NUM_PARTIES)]
    nums = [i+1 for i in range(k)]
    shares = [shamir_share(x, t, n) for x in nums]
    share_groups = list(zip(*shares))

    for p,xs in zip(parties, share_groups):
        p.round1(parties, xs, t)

    round_num = 2
    while not parties[0].is_done:
        for p in parties:
            p.roundn(round_num)   
 
        round_num += 1

    output_shares = [p.output for p in parties]

    print(output_shares)
    
    print('Reconstruction, with all shares:', reconstruct(output_shares))
    print('Reconstruction, with 3 shares:', reconstruct(output_shares[:3]))
    print('Reconstruction, with 2 shares:', reconstruct(output_shares[:2]))
    return nums, output_shares

In [6]:
# TEST CASE for question 4
nums, output_shares = run_list_prod()

print(nums)
print(np.prod(nums))

assert reconstruct(output_shares) == np.prod(nums)
assert reconstruct(output_shares[:3]) == np.prod(nums)
assert reconstruct(output_shares[:2]) != np.prod(nums)

[(GF(1, order=8191), GF(1635, order=8191)), (GF(2, order=8191), GF(5362, order=8191)), (GF(3, order=8191), GF(3110, order=8191)), (GF(4, order=8191), GF(3070, order=8191)), (GF(5, order=8191), GF(5242, order=8191)), (GF(6, order=8191), GF(1435, order=8191))]
Reconstruction, with all shares: 120
Reconstruction, with 3 shares: 120
Reconstruction, with 2 shares: 6099
[1, 2, 3, 4, 5]
120
